# Final Project Baseline: House Prices- Advanced Regression Techniques
### Author: Radia Abdul Wahab, Parham Motameni, Jun Qian
### Date: Fall 2021
### Course: w207 Machine Learning



# 

In [9]:
# This tells matplotlib not to try opening a new window for each plot.
%matplotlib inline


In [10]:
# Import libraries.

from sklearn.metrics import r2_score
import random
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from matplotlib.ticker import MultipleLocator
from sklearn.pipeline import Pipeline
from sklearn.datasets import fetch_openml
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LinearRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
import scipy.io as sio


In [11]:
# lock the seed to have repeatable results
random.seed(100)

In [12]:
# load datatsets and labels
def load_dataset():
    # X_train is sparse matrix others are not
    X_train = sio.mmread("X_train.mtx").todense()
    X_test = pd.read_csv('X_test.csv')
    y_train, y_test=  pd.read_csv('y_train.csv'),  pd.read_csv('y_test.csv')
    return X_train, y_train, X_test, y_test


# Helper methods

In [13]:
def predict_and_display_score(regressor, X_test, y_test):
    y_pred = regressor.predict(X_test)
    print(
        f'r2_score of {type(regressor).__name__}: {r2_score(y_test, y_pred)}')
    return y_pred

def get_side_by_side_comparison(y_pred, y_test ):
    comp = pd.concat([pd.DataFrame(y_pred), y_test], axis=1)
    return comp


# Training the Multiple Linear Regression model on the Training set

In [14]:
def run_regression(regressor, X_train, y_train, X_test, y_test):
    regressor.fit(X_train, y_train)
    predict_and_display_score(regressor, X_test, y_test)


In [15]:
def run_all():
    regressors = [LinearRegression(),
                    DecisionTreeRegressor(random_state=0),
                    RandomForestRegressor(n_estimators=10, random_state=0)
                ]
    X_tr, y_tr, X_tst, y_tst = load_dataset()
    [run_regression(r, X_tr, y_tr.values.ravel(), X_tst, y_tst.values.ravel())
     for r in regressors]


## Predicting and displaying set results for Regression models

In [16]:
run_all()

r2_score of LinearRegression: 0.8775862774364419
r2_score of DecisionTreeRegressor: 0.8225349679881327
r2_score of RandomForestRegressor: 0.8902441767129868
